In [73]:
import sys

from hfinger.analysis import hfinger_analyze

import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
from difflib import SequenceMatcher
import jellyfish

In [74]:
# Fonction de calcul de distance avec sequenceMatcher

def distance(f1,f2):
    return(1-SequenceMatcher(None,f1,f2).ratio()) # isjunk=None (no element ignored), .ratio give float between [0,1]

In [28]:
# Fonction de calcul de distance avec levenshtein
def distance(f1,f2):
    return(jellyfish.levenshtein_distance(f1,f2))


In [15]:
#to use jaro distance
def distance(f1,f2):
    return(jellyfish.jaro_similarity(f1,f2))

In [5]:
#Use damerau levenstein distance
def distance(f1,f2):
    return(jellyfish.damerau_levenshtein_distance(f1,f2))

In [75]:
# Fonction d'affichage du dendogramme
def plot_dendrogram(model, **kwargs):

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
                counts[i] = current_count

                linkage_matrix = np.column_stack(
                [model.children_, model.distances_, counts]
                ).astype(float)

                # Plot the corresponding dendrogram
                dendrogram(linkage_matrix, **kwargs)

In [ ]:
# Formatage des empreintes
listOfFingerprints=[]
listWithAllTaggedFingerprints=[]

# Attention remplacer liste_Arguments par sys.argv si en dehors de Jupiter
liste_Arguments = ["blabla","GlobeImposter","legitimeFirefox","legitimeSafari"]

for z in liste_Arguments[1:]:
    filePath = "files/"+z
    with open(filePath, "r") as file:
        listOfFingerprintsInFile = []
        for i in file:
            fp = i.split("|")
            fp = list(map(lambda x : x if x != '' else 0, fp))
            fp.pop()
            fp = "|".join(str(v) for v in fp)
            listOfFingerprintsInFile.append(fp)
    listOfFingerprints.extend(listOfFingerprintsInFile)
    listWithAllTaggedFingerprints.extend([[y, filePath] for y in listOfFingerprintsInFile])
    

# Création de la matrice de distances
listDistances=[] # Chaque liste contient l'ensemble des distances par rapport a une empreinte.
for fingerprint in listOfFingerprints:
    listOfDist=[]
    for fingerprint2 in listOfFingerprints:
        listOfDist.append(distance(fingerprint,fingerprint2))                    
    listDistances.append(listOfDist)

In [ ]:
# Création du modèle
model = AgglomerativeClustering(distance_threshold=None, n_clusters=3) # n_cluster= number of cluster to find, if not none distance must be none. 
model = model.fit(listDistances) 



#print(model.n_clusters_)
#print(model.labels_)
for k in range(model.n_clusters_):
    print("\ncluster number "+str(k))
    for p in range(len(listOfFingerprints)):
        if model.labels_[p]==k:
            print(listOfFingerprints[p]+' '+listWithAllTaggedFingerprints[p][1])

In [ ]:
# Affichage
plt.title("Dendogramme de Regroupement Hierarchique")
#plot_dendrogram(model, truncate_mode="level", p=10) # plot the top ten levels of the dendrogram
plt.xlabel("Nombre de points dans un noeud (ou index de point s'il n'y a pas de parenthèse)")
plt.ylabel("Distance entre les clusters")
#plt.show()